In [1]:
#!pip install python-igraph

# Load the graphs

In [2]:
#@title  { form-width: "30%" }
import tqdm as tq
import igraph as ig 
import os

datapath = "/Volumes/GoogleDrive/Il Mio Drive/TUDatasets" #@param {type:"string"}
dataset = "Kidney_9.2" #@param ["Mutag", "LFR", "MREG", "Kidney_9.2", "COBREpos"] {allow-input: true}
format = "graphml" #@param ["graphml", "edgelist"] {allow-input: true}
path = f'{datapath}/{dataset}/{format}'
filenames = os.listdir(path)
graphs = []
for f in tq.tqdm(filenames):
  if f.endswith(format):
    g = ig.load(os.path.join(path,f))
    g["name"] = f.split('.')[0]
    graphs += [g]

100%|██████████| 299/299 [00:03<00:00, 99.23it/s] 


# Read the labels

In [12]:
import pandas as pd
import numpy as np
dfl = pd.read_csv(f'{datapath}/{dataset}/{dataset}.txt', sep='\t')
last_column = dfl.iloc[:,[0] + [-1]]
#y = [last_column[last_column['Samples'] == g["name"]].iloc[:,-1:].values[0] for g in graphs]
print(list(last_column[last_column['Samples'] == g["name"]] for g in graphs))
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(np.ravel(y))
y = le.transform(np.ravel(y))
assert len(y) == len(graphs)
from collections import Counter
Counter(y)

[Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [Samples, labels]
Index: [], Empty DataFrame
Columns: [S

AssertionError: 

# Convert edges to nodes

In [162]:
def dualgraph(g):
    def add_vertex_with_attrs(graph, attrs):
        n = graph.vcount()
        graph.add_vertices(1)
        for key, value in attrs.items():
            graph.vs[n][key] = value

    dg = ig.Graph(directed=True) if g.is_directed() else ig.Graph() 
    for edge in g.es:
        edge_attrs = edge.attributes()
        source, target = edge.source, edge.target
        edge_attrs["name"] = f'{edge.source}_{edge.target}'
        add_vertex_with_attrs(dg, edge_attrs)
    outedges = []
    for node in dg.vs:
        source, target = [int(a) for a in node['name'].split('_')]
        outedges += [f'{target}_{e.source}' if e.target == target else f'{target}_{e.target}' for e in g.es.select(_source=target)]
        outedges = list(set(outedges))
    newoutedges = []
    for i in range(len(outedges)):
        s, t = [int(a) for a in outedges[i].split('_')]
        if not f'{t}_{s}' in outedges[i:]:
            newoutedges.append(outedges[i])
    print(node.index, target, newoutedges)
    toadd = []
    for e in newoutedges:
        s, t = [int(a) for a in e.split('_')] 
        try:
            vs = dg.vs.find(name_eq=f'{s}_{t}')
        except:
            vs = dg.vs.find(name_eq=f'{t}_{s}')
        #toadd.append((node.index,vs.index)) if not dg.are_connected(node.index,vs.index) else None
        toadd.append((node.index,vs.index))
    print(toadd)
    [dg.add_edge(*edge) for edge in toadd if not dg.are_connected(edge[0], edge[1])]
    return dg

In [163]:
import igraph as ig
graph = ig.Graph.Full(n=3)
dg = dualgraph(graph)
print(graph, graph.vcount(), graph.ecount())
print(dg, dg.vs["name"], dg.vcount(), dg.ecount())
for e in dg.es:
    print("Edge; ", e.source, e.target)

2 2 ['1_0', '2_0', '2_1']
[(2, 0), (2, 1), (2, 2)]
IGRAPH U--- 3 3 --
+ edges:
0--1 0--2 1--2 3 3
IGRAPH UN-- 3 3 --
+ attr: name (v)
+ edges (vertex names):
0_1--1_2, 0_2--1_2, 1_2--1_2 ['0_1', '0_2', '1_2'] 3 3
Edge;  0 2
Edge;  1 2
Edge;  2 2


In [3]:
def add_vertex_with_attrs(graph, attrs):
    n = graph.vcount()
    graph.add_vertices(1)
    for key, value in attrs.items():
        graph.vs[n][key] = value
        
dualgraphs = []
for g in graphs:
    dg = ig.Graph(directed=True) if g.is_directed() else ig.Graph() 
    for edge in g.es:
        edge_attrs = edge.attributes()
        source, target = edge.source, edge.target
        edge_attrs["name"] = f'{edge.source}_{edge.target}'
        add_vertex_with_attrs(dg, edge_attrs)
    for node in dg.vs:
        source, target = node['name'].split('_')
        if g.is_directed():
            outnodes = g.incident(int(target), mode = "out")
        else:
            outnodes = g.incident(int(target), mode = "all")
        [dg.add_edge(node,n) for n in outnodes]
    dualgraphs.append(dg)
dualgraphs[0].es

In [48]:
dualgraphs[1].ecount()

IndexError: list index out of range